In [1]:
import pandas as pd
import numpy as np
import csv 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
# Read csv
asthma_df = pd.read_csv('astma.csv')

# Define cols
colnames = ["Index","SubjectNr","DayNo","weekday","dayType","Age","sex","weight","height","BMI_SDS","ethnicity",
           "school_year_final","sportsyesno","urbanisation","PedsQL_score_baseline","stepsTotalDaily","steps_hour_max",
           "steps00","steps01","steps02","steps03","steps04","steps05","steps06","steps07","steps08","steps09","steps10",
           "steps11","steps12","steps13","steps14","steps15","steps16","steps17","steps18","steps19","steps20","steps21",
           "steps22","steps23","HR05Perc","HR95Perc","HRMinSleep","HRMaxSleep","AVGHR_daily","AVGHR_sleep","AVGHR_wake",
           "HR00","HR01","HR02","HR03","HR04","HR05","HR06","HR07","HR08","HR09","HR10","HR11","HR12","HR13","HR14","HR15",
           "HR16","HR17","HR18","HR19","HR20","HR21","HR22","HR23","wear05H","wear16H","wear24H","BODY_TEMPERATURE_DEG_C",
           "DIASTOLIC_BLOOD_PRESSURE_MMHG","HEART_PULSE_BPM","SYSTOLIC_BLOOD_PRESSURE_MMHG","WEIGHT_KG","awakeDuration",
           "lightSleepDuration","deepSleepDuration","wakeUpCount","sleeptime","waketime","fvc_best","fev1_best","pef_best",
           "grade_fev1","grade_fvc","predicted_fvc_best","predicted_fev1_best","predicted_fev1_ratio_best","fev1_ratio_best",
           "fev1_percentage","fvc1_percentage","fev1_ratio_percentage","school_yes_no","screentime","FG","FHX","FHN","TG","TN",
           "TX","SQ","SP","DR","RH","RHX"]

# This removes the "" in health csv file
rows = []
with open("healthy_share.csv") as file:
    for l in csv.reader(file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True):
        rows.append(''.join(l).split(','))

# Removes colum list from data rows
rows.pop(0)

healthy_df = pd.DataFrame(rows, columns=colnames)

In [8]:
display(healthy_df)

Index SubjectNr DayNo weekday    dayType Age       sex weight height BMI_SDS        ethnicity   school_year_final sportsyesno            urbanisation PedsQL_score_baseline stepsTotalDaily steps_hour_max steps00 steps01 steps02 steps03 steps04 steps05 steps06 steps07 steps08 steps09 steps10 steps11 steps12 steps13 steps14 steps15 steps16 steps17 steps18 steps19 steps20 steps21 steps22 steps23 HR05Perc HR95Perc HRMinSleep HRMaxSleep AVGHR_daily AVGHR_sleep AVGHR_wake         HR00         HR01         HR02         HR03         HR04         HR05         HR06         HR07         HR08         HR09         HR10         HR11         HR12         HR13         HR14         HR15   HR16         HR17         HR18         HR19         HR20         HR21         HR22         HR23 wear05H wear16H wear24H BODY_TEMPERATURE_DEG_C DIASTOLIC_BLOOD_PRESSURE_MMHG HEART_PULSE_BPM SYSTOLIC_BLOOD_PRESSURE_MMHG WEIGHT_KG awakeDuration lightSleepDuration deepSleepDuration wakeUpCount sleeptime  waketime  \
0        1   2853189     0   "Thu"  "holiday"   2  "Female"     12     92    -1.8  "Other / Mixed"          "Day care"        "No"   "Extremely urbanised"      97.8260869565217            2806           1992      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA       7      NA      NA      NA      NA     480    1992     156      51      61      39      NA      20       56      123         NA         NA          96          NA         97           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA     NA        115.5        106.5          109           NA           56           NA           92       0      44      33                     NA                            NA              NA                           NA        NA            NA                 NA                NA          NA        NA        NA   
1        2   2853189     1   "Fri"  "holiday"   2  "Female"     12     92    -1.8  "Other / Mixed"          "Day care"        "No"   "Extremely urbanised"      97.8260869565217            2564           1205     151      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA      32       9      58      NA      NA      NA    1205     599      NA      NA      NA       52      163         NA         NA         126         119        128          119           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA     NA           NA           NA        126.5        113.6          145           NA           NA      20      38      29                     NA                            NA              NA                           NA        NA            NA                 NA                NA          NA        NA        NA   
2        3   2853189     2   "Sat"  "holiday"   2  "Female"     12     92    -1.8  "Other / Mixed"          "Day care"        "No"   "Extremely urbanised"      97.8260869565217             521            369      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA      NA     369      78      NA      NA      NA      NA      NA      NA      74      NA      NA      NA      NA       71      137         NA         NA         103          NA        103           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA           NA          107           NA           NA           NA           NA     NA           NA        102.5  100.3333333           NA           NA           NA           NA       0      25      17                     NA                            NA              NA                           NA        NA            NA                 NA                NA          NA        NA        NA   
3        

In [9]:
display(asthma_df)

Unnamed: 0  SubjectNr  DayNo weekday  dayType  stepsTotalDaily  stepsTotalDetailed  steps00  steps01  steps02  steps03  steps04  steps05  steps06  steps07  steps08  steps09  steps10  steps11  steps12  steps13  steps14  steps15  steps16  steps17  steps18  steps19  steps20  steps21  steps22  steps23  HR05Perc  HR95Perc     HrAvgWake          HR00          HR01          HR02          HR03          HR04          HR05          HR06          HR07          HR08          HR09          HR10          HR11          HR12          HR13          HR14          HR15          HR16          HR17          HR18          HR19          HR20          HR21          HR22          HR23  awakeDuration  lightSleepDuration  deepSleepDuration  wakeUpCount  HRAvgSleep  HRMinSleep  HRMaxSleep  activity_score_parent school_yes_no  activity_score_child            screentime  sleep_score_child bedtimeReport waketimeReport  fvc_best  fev1_best  pef_best grade_fev1 grade_fvc  Technique  fev1_ratio_best  \
0              1    1771258      0     Tue   school           3723.0              3723.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN     63.0    137.0      NaN      NaN    605.0   1640.0      NaN     82.0    451.0    245.0    135.0    160.0    164.0     41.0      NaN      68.0     120.0  8.689362e+09           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN  1.200000e+02           NaN           NaN           NaN           NaN           NaN  8.985714e+09  8.342857e+09  8.787500e+04  9.150000e+01  9.337500e+04  7.716667e+09  7.866667e+09          660.0             16800.0            16440.0          1.0        75.0        61.0        93.0                    NaN           NaN                   NaN                   NaN                NaN           NaN            NaN   3232.00     2381.0   4334.00          D         B        3.0         0.736696   
1              2    1771258      1     Wed   school          10015.0             10015.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN    607.0    580.0    325.0    180.0    322.0    181.0    491.0    161.0     10.0    744.0   1146.0   4355.0    722.0    142.0     49.0      NaN      82.0     168.0  1.326242e+09  7.360000e+01  7.528571e+09  7.075000e+01  9.200000e+01  7.600000e+01  7.033333e+09  8.233333e+09  7.800000e+01  9.900000e+01  1.130000e+02  1.298000e+02  9.516667e+09  9.166667e+09           NaN  1.085000e+02  8.525000e+01  9.340000e+01  9.588889e+09  1.018333e+09  1.423074e+09  1.078154e+09  1.042000e+02  9.183333e+09  9.300000e+01          180.0             22200.0            11760.0          0.0        84.0        67.0       121.0                    8.0            Ja                   8.0  B. 30 tot 60 minuten                9.0      15:34:00       13:04:00   3261.00     2257.0   4509.00          A         A        3.0         0.692119   
2              3    1771258      2     Thu   school           3811.0              3811.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN     63.0    561.0    159.0    356.0     26.0    591.0    109.0    114.0    456.0    155.0    727.0     46.0    139.0    210.0      6.0     93.0      NaN      71.0     122.0  9.030597e+09  7.916667e+09  7.316667e+09  8.033333e+09  7.800000e+01  7.614286e+09  7.466667e+09  7.814286e+09  9.850000e+01  1.034286e+09  1.005714e+09  1.096667e+09  1.006667e+09  1.050000e+02  1.090000e+02  8.650000e+01  8.866667e+09  1.028000e+02  7.820000e+01  8.520000e+01  9.400000e+01  1.000000e+02  8.925000e+01  9.650000e+01  8.200000e+01          300.0             19260.0            12360.0          1.0        76.0        64.0       129.0                    5.0            Ja                   6.0  B. 30 tot 60 minuten                9.0      15:19:00       13:09:00   3451.00     2567.0   4717.00          B         B        3.0         0.743842   
3              4    1771258      3     Fri   s

# Preprocess

In [ ]:
# Renaming columns (allignment)
asthma_df = asthma_df.rename(columns={'Gender': 'sex'})
asthma_df = asthma_df.rename(columns={'Weight': 'weight'})
asthma_df = asthma_df.rename(columns={'Height': 'height'})

# What grade they are in
healthy_df = healthy_df.rename(columns={'school_year_final': 'school_year'})
asthma_df = asthma_df.rename(columns={'school_year_edit': 'school_year'})

# Wheter they do a sport
asthma_df = asthma_df.rename(columns={'Sports': 'sportsyesno'})

# Note there are 2 urbanisations
asthma_df = asthma_df.rename(columns={'urbanisation2': 'urbanisation'})

# pedsql
asthma_df = asthma_df.rename(columns={'BASELINE_PedsQL_score': 'PedsQL_score_baseline'})

# NOTICE THAT WEAR05 WEAR16 AND WEAR24 ARE PERCENTAGES OF WORN (HEALTHY_DF)
# NOTICE THAT WEAR01, WEAR02 ARE YES OR NO  (ASTHMA_DF)
# TODO WE CAN ALLIGN THESE

# No temperature in asthma_df but is in healthy_df?
# TODO drop in healthy 

# No blood preassure in asthma_df but is in healthy_df?
# TODO drop in healthy

# Note that WEIGHT_KG is a daily meassurement in healthy_df
# in asthma_df this meassurement is only taken at begin and end of the study period

# In healthy_df the value sleeptime is the time they went to bed this isnt in asthma_df
# bedtimeReport waketimereport?

# continue at: predicted_fvc_best

In [10]:
# Removing columns
healthy_df = healthy_df.drop('Index', axis=1)

# TODO delete each column in asthma_df that isnt already in healthy_df

In [ ]:
# one-hot encoding
one_hot_asthma = asthma_df.get_dummies(df[['weekday', 'dayType']])
one_hot_healthy = healthy_df.get_dummies(df[['weekday', 'dayType']])

asthma_df = pd.concat([asthma_df, one_hot_asthma], axis=1)
healthy_df = pd.concat([healthy_df, one_hot_healthy], axis=1)

# Model

In [5]:
# Create the feature matrix X by concatenating the two dataframes
X = pd.concat([asthma_df, healthy_df], axis=0)

# Create the target variable y, where 1 represents asthmatic patients and -1 represents healthy patients
y = pd.Series([1] * len(asthma_df) + [0] * len(healthy_df))

# Split the data into training and testing sets using a 70-30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the One-Class SVM model to the training data
model = OneClassSVM().fit(X_train[y_train == 1])

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the performance of the model
accuracy = (y_pred == y_test).mean()
print("Accuracy: {:.2f}".format(accuracy))

C:\Users\danie\AppData\Local\Temp\ipykernel_2800\3498351387.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  model = OneClassSVM().fit(X_train[y_train == 1])


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).